In [3]:
!nvidia-smi || echo "No GPU available"
!pip show transformers

Fri Jul 18 07:27:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

assert torch.cuda.is_available(), "CUDA is not available"
device = torch.device("cuda:0")

print(f"Using device: {torch.cuda.get_device_name(0)}")

Using device: Tesla T4


In [5]:
!pip install -q \
    transformers==4.40.2 \
    peft==0.11.0 \
    accelerate==0.30.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 86.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.9 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from peft import PeftModel
from datasets import load_dataset
import pandas as pd

In [7]:
BASE_MODEL = "/kaggle/input/phi-3/pytorch/phi-3.5-mini-instruct/2"
LORA_ADAPTER = "/kaggle/input/final-results/lora_phi3_stage2/checkpoint-6500"
TEST_PATH = "/kaggle/input/goedel-machines-x-iitm-clinical-llm-challenge"
OUT_CSV = "/kaggle/working/submission.csv"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, local_files_only=True)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map={"": 0},
    trust_remote_code=True,
    local_files_only=True
)

model = PeftModel.from_pretrained(base_model, LORA_ADAPTER)
model.eval()
model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: LoraConfig.__init__() got an unexpected keyword argument 'corda_config'

In [ ]:
from datasets import load_dataset
import pandas as pd

from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(BASE_MODEL)
gen_config.max_new_tokens = 10

def predict_mcq(question, options):
    prompt = f"{question}\n" + "\n".join([f"({k}) {v}" for k, v in options.items()]) + "\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, generation_config=gen_config)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    for opt in options:
        if f"({opt})" in decoded:
            return opt
    return "A" 

all_preds = []
for level in ["easy", "medium", "hard"]:
    dataset = load_dataset(
        "json", 
        data_files=f"/kaggle/input/goedel-machines-x-iitm-clinical-llm-challenge/train_{level}.jsonl", 
        split="train"
    )
    for ex in dataset:
        pred = predict_mcq(ex["question"], ex["options"])
        all_preds.append({"id": ex["id"], "answer": pred})

submission_df = pd.DataFrame(all_preds)
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ Submission saved at /kaggle/working/submission.csv")


In [ ]:
def evaluate(file_path):
    dataset = load_dataset("json", data_files=file_path, split="train")
    correct = 0
    for example in dataset:
        pred = predict_answer(example["question"], example["options"])
        if pred == example["answer"]:
            correct += 1
    accuracy = correct / len(dataset)
    print(f"{file_path} — Accuracy: {accuracy:.4f}")
    return accuracy

print("done")

In [ ]:
!pip install -q -U datasets

In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
from datasets import load_dataset

acc_easy = evaluate("/kaggle/input/goedel-machines-x-iitm-clinical-llm-challenge/sample_test_easy.jsonl")
acc_medium = evaluate("/kaggle/input/goedel-machines-x-iitm-clinical-llm-challenge/sample_test_medium.jsonl")
acc_hard = evaluate("/kaggle/input/goedel-machines-x-iitm-clinical-llm-challenge/sample_test_hard.jsonl")

overall = 0.30 * acc_easy + 0.30 * acc_medium + 0.40 * acc_hard
print(f"Overall Score: {overall:.4f}")